In [174]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline

In [175]:
#Open the problem file
problem = open("test_problem.uai", "r")

#Define problem variables
preamble = problem.readline().rstrip('\n')
numOfvars = 0
cardinalities = {}
numOfCliques = 0
cliques = {}
cliques_size = {}
sizeOfCPT = []
CPT_weights = []

#Read the file into the problem variables
if preamble == "MARKOV" or preamble == "BAYES":
    
    #Read the cardinalities
    numOfvars = int(problem.readline())
    cardinalities = [int(i) for i in problem.readline().split(' ')]
    
    #Read the cliques
    numOfCliques = int(problem.readline())
    for i in range(numOfCliques):
        temp_clique = [int(i) for i in problem.readline().split(' ')]
        cliques_size[i] = temp_clique[0]
        cliques[i] = temp_clique[1:]
    
    #Read the conditional probability table
    for i in range(numOfCliques):
        #Assumes blank line - should be skipped
        skipLine = problem.readline()
        
        sizeOfCPT.append(int(problem.readline()))
        temp_CPT = [[float(i) for i in problem.readline().split(' ')[1:]]]
        linesLeftToRead = int(sizeOfCPT[-1]/len(temp_CPT[0])) - 1
        for j in range(linesLeftToRead):
            temp_CPT.append([float(i) for i in problem.readline().split(' ')[1:]])
        #CPT.append(temp_CPT)
        CPT_weights.append(np.concatenate(temp_CPT).tolist())
else:
    print("The file provided does not correctly specify the problem type. Supported preambles are 'BAYES' and 'MARKOV'.")

#Close the problem file
problem.close()

In [176]:
CPT_weights

[[0.21, 0.333, 0.457, 0.811, 0.0, 0.189]]

In [177]:
#Considering cliques construct CPT states
def construct_CPT_states(clique):
    temp_CPT_states = []
    #Base case
    if len(clique) == 1:
        #print("Base case")
        for cardinality in range(cardinalities[clique[0]]):
            temp_CPT_states.append([cardinality])
        #print(temp_CPT_states)
        return temp_CPT_states
    #Recursive case
    else:
        #print("Recursive case")
        CPT_states = construct_CPT_states(clique[1:])
        for cardinality in range(cardinalities[clique[0]]):
            for row in CPT_states:
                temp_CPT_states.append(np.concatenate([[cardinality], row]).tolist())
        #print(temp_CPT_states)
        return temp_CPT_states

CPT_states = []
for key in cliques.keys():
    CPT_states.append(construct_CPT_states(cliques[key]))

In [178]:
CPT_states

[[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0], [2, 1]]]

In [179]:
#Open the query (evidence) file 
query = open("test_problem.uai.evid", "r")

#Read the query file into the query variables
query_vec = [int(i) for i in query.read().split(' ')]
numOfObservedVars = query_vec[0]
observations = {}
for i in range(numOfObservedVars):
    observations[query_vec[2*i+1]] = query_vec[2*i+2]

#Close the query file
query.close()

In [180]:
observations

{1: 1}

In [181]:
#Produce encodings for the indicator variables 
# of the elementary assignments (X_i,j)
indicatorVars = {}
encoding = 1
for var in range(numOfvars):
    indicatorVars[var] = []
    for cardinality in range(cardinalities[var]):
        indicatorVars[var].append(encoding)
        encoding = encoding + 1

#Produce encodings for each conditional probability weight in every CPT
weightIndicatorVars = {}
for c in range(numOfCliques):
    weightIndicatorVars[c] = []
    for w in range(sizeOfCPT[c]):
        weightIndicatorVars[c].append(encoding)
        encoding = encoding + 1

numOfFormulaVars = encoding - 1
F = []
#Generate clauses for elementary assignements
#For each variable constructs the corresponding clauses
for var in range(numOfvars):
    #Genrate clause for possible states of the variable i
    clause = []
    for cardinality in range(cardinalities[var]):
        clause.append(indicatorVars[var][cardinality])
    F.append(clause)
    
    #Generate clause for every different pair of possible states
    # of the variable i
    for n in range(cardinalities[var]):
        for m in range(n+1, cardinalities[var]):
            clause = []
            clause.append(-indicatorVars[var][n])
            clause.append(-indicatorVars[var][m])
            if len(clause) != 0:
                F.append(clause)
        
#Genrate clauses for the CPT
# Incorporates following optimizations:
# 1. Determinism (zero weights)
for c in range(numOfCliques):
    for i in range(sizeOfCPT[c]):
        clause = []
        #Take adventage of the determinism
        if weightIndicatorVars[c][i] == 0:
            for j in range(len(cliques[c])):
                clause.append(-indicatorVars[cliques[c][j]][CPT_states[c][i][j]])
        else:
            for j in range(len(cliques[c])):
                clause.append(-indicatorVars[cliques[c][j]][CPT_states[c][i][j]])
                F.append([indicatorVars[cliques[c][j]][CPT_states[c][i][j]],-weightIndicatorVars[c][i]])
            clause.append(weightIndicatorVars[c][i])
        F.append(clause)
numOfClauses = len(F)

In [182]:
F

[[1, 2, 3],
 [-1, -2],
 [-1, -3],
 [-2, -3],
 [4, 5],
 [-4, -5],
 [1, -6],
 [4, -6],
 [-1, -4, 6],
 [1, -7],
 [5, -7],
 [-1, -5, 7],
 [2, -8],
 [4, -8],
 [-2, -4, 8],
 [2, -9],
 [5, -9],
 [-2, -5, 9],
 [3, -10],
 [4, -10],
 [-3, -4, 10],
 [3, -11],
 [5, -11],
 [-3, -5, 11]]

In [187]:
#Generate CNF file
CNF_file = open("test_problem.cnf", "w")
CNF_file.write("p cnf " + str(numOfFormulaVars) + " " + str(numOfClauses) + "\n")
for clause in F:
    CNF_file.write(" ".join(map(str, clause))+" 0\n")
CNF_file.close()

In [184]:
#Generate weights
weights = {}
#For state indicator variables
for var in range(numOfvars):
    for cardinality in range(cardinalities[var]): 
        weights[indicatorVars[var][cardinality]] = 1 
        weights[-indicatorVars[var][cardinality]] = 1 
#For weight indicator variables
for c in range(numOfCliques):
    for w in range(sizeOfCPT[c]):
        weights[weightIndicatorVars[c][w]] = CPT_weights[c][w]
        weights[-weightIndicatorVars[c][w]] = 1

In [185]:
weights

{1: 1,
 -1: 1,
 2: 1,
 -2: 1,
 3: 1,
 -3: 1,
 4: 1,
 -4: 1,
 5: 1,
 -5: 1,
 6: 0.21,
 -6: 1,
 7: 0.333,
 -7: 1,
 8: 0.457,
 -8: 1,
 9: 0.811,
 -9: 1,
 10: 0.0,
 -10: 1,
 11: 0.189,
 -11: 1}

In [190]:
#Generate weights file
weights_file = open("test_problem.weights", "w")
weights_file.write("p cnf " + str(numOfFormulaVars) + "\n")
for var in weights.keys():
    weights_file.write("w " + str(var) + " " + str(weights[var]) + " 0\n")
weights_file.close()